In [1]:
from skshapes.morphing import ElasticMetric
from skshapes.loss import OptimalTransportLoss
from skshapes.data import read
from skshapes.tasks import DistanceMatrix

In [7]:
# Load some shapes
import os

datafolder = "/home/GitHub/scikit-shapes-draft/data/SCAPE/decimated/"
datafolder = "data/SCAPE_low_resolution"


shapes = [read(datafolder + "/" + f"mesh{i:03d}.ply") for i in range(10)]

dm = DistanceMatrix(
    model=ElasticMetric(),
    loss=OptimalTransportLoss(),
    verbose=1,
    n_iter=3,
    n_steps=5,
)

distance_matrix = dm.fit_transform(shapes=shapes)

Fitting shapes 0 and 1...
Loss value at iteration 0 : 0.007982032373547554
Loss value at iteration 1 : 0.0035379347391426563
Loss value at iteration 2 : 0.0008693138370290399
Fitting shapes 0 and 2...
Loss value at iteration 0 : 0.029302038252353668
Loss value at iteration 1 : 0.016969991847872734
Loss value at iteration 2 : 0.009617364034056664
Fitting shapes 0 and 3...
Loss value at iteration 0 : 0.036833472549915314
Loss value at iteration 1 : 0.012755298987030983
Loss value at iteration 2 : 0.006045044399797916
Fitting shapes 0 and 4...
Loss value at iteration 0 : 0.05379059165716171
Loss value at iteration 1 : 0.019368203356862068
Loss value at iteration 2 : 0.013924786821007729
Fitting shapes 0 and 5...
Loss value at iteration 0 : 0.01185077615082264
Loss value at iteration 1 : 0.00842280499637127
Loss value at iteration 2 : 0.007331870961934328
Fitting shapes 0 and 6...
Loss value at iteration 0 : 0.07037040591239929
Loss value at iteration 1 : 0.018142174929380417
Loss value at

In [19]:
from sklearn.cluster import SpectralClustering
import numpy as np

affinity_matrix = np.exp(- (distance_matrix / 0.001) ** 2)

clustering = SpectralClustering(affinity="precomputed", n_clusters=2).fit(affinity_matrix)
print(clustering.labels_)

[0 0 0 0 0 0 1 0 1 0]


In [1]:
from skshapes.preprocessing import (
    LandmarkSetter,
    Pipeline,
    Decimation,
    AffineTransformation,
)
from skshapes.data import read
import os

# Read the shapes
datafolder = "/home/GitHub/scikit-shapes-draft/data/SCAPE/decimated/"
shapes = [read(datafolder + "/" + f"aligned_mesh{i:03d}.ply") for i in range(4)]

# Create a pipeline of preprocessing steps
preprocessing = Pipeline(
    steps=[
        LandmarkSetter(landmarks=[[0, 1], [2, 3], [4, 5], [6, 70]], by_indices=True),
        Decimation(target_reduction=0.95),
        AffineTransformation(matrix=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    ]
)

# Apply the preprocessing pipeline to the shapes and get the dataset
dataset = preprocessing.fit_transform(shapes=shapes)

for i in range(len(dataset)):
    print("Size of shape {} : {}".format(i, dataset[i].points.shape))
    print("Landmarks of shape {} : {}".format(i, dataset.landmarks[i]), end="\n\n")

Size of shape 0 : torch.Size([518, 3])
Landmarks of shape 0 : tensor([[ 0.0156, -0.0755,  0.2701],
        [ 0.0190,  0.0418,  0.2677]])

Size of shape 1 : torch.Size([446, 3])
Landmarks of shape 1 : tensor([[0.0314, 0.0201, 0.2025],
        [0.0319, 0.0535, 0.1964]])

Size of shape 2 : torch.Size([465, 3])
Landmarks of shape 2 : tensor([[-0.0111,  0.0480,  0.1389],
        [-0.0108, -0.0148,  0.1165]])

Size of shape 3 : torch.Size([444, 3])
Landmarks of shape 3 : tensor([[ 0.0917, -0.3005,  0.1144],
        [ 0.0360,  0.2349,  0.0129]])



In [8]:
t = dataset.landmarks[0]
list(t)

[tensor([ 0.0156, -0.0755,  0.2701]), tensor([0.0190, 0.0418, 0.2677])]

In [68]:
import pyvista
import numpy as np

polydata = pyvista.read(os.path.join(datafolder, f"aligned_mesh008.ply"))
old_points = polydata.points.copy()
polydata.decimate_pro(0.999, preserve_topology=True, inplace=True)
new_points = polydata.points.copy()

[
    np.where(np.all(new_points[i] == old_points, axis=1))[0][0]
    for i in range(len(new_points))
]

[4,
 5,
 8,
 9,
 10,
 12,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 24,
 25,
 26,
 27,
 30,
 32,
 34,
 35,
 36,
 39,
 41,
 42,
 44,
 45,
 46,
 49,
 50,
 53,
 55,
 56,
 57,
 60,
 62,
 64,
 66,
 67,
 69,
 70,
 71,
 73,
 74,
 75,
 77,
 79,
 80,
 81,
 85,
 86,
 87,
 88,
 89,
 90,
 94,
 95,
 96,
 97,
 98,
 100,
 101,
 104,
 105,
 106,
 108,
 110,
 111,
 112,
 113,
 115,
 116,
 119,
 125,
 135,
 138,
 139,
 141,
 145,
 151,
 152,
 155,
 159,
 165,
 166,
 169,
 173,
 177,
 178,
 181,
 182,
 185,
 187,
 199,
 204,
 213,
 214,
 216,
 217,
 219,
 240,
 241,
 243,
 244,
 247,
 252,
 259,
 266,
 267,
 270,
 273,
 279,
 280,
 284,
 290,
 296,
 298,
 300,
 313,
 319,
 328,
 344,
 351,
 359,
 366,
 367,
 378,
 390,
 404,
 414,
 424,
 443,
 451,
 470,
 486,
 487,
 495,
 499,
 500,
 504,
 512,
 524,
 529,
 537,
 539,
 540,
 545,
 546,
 552,
 556,
 566,
 587,
 588,
 602,
 608,
 609,
 613,
 618,
 619,
 620,
 624,
 627,
 629,
 630,
 632,
 633,
 634,
 637,
 639,
 644,
 647,
 650,
 651,
 653,
 654,
 656,
 657,
 6

In [5]:
shape = shapes[0]
pv_shape = shape.to_pyvista()

from skshapes.data import Shape

shape2 = Shape.from_pyvista(pv_shape)

In [9]:
shape.edges

tensor([[ 864,  871,  842,  ..., 1558, 1560, 1561],
        [ 871,  842,  864,  ..., 1559, 1561, 1559]])

In [65]:
print(new_points.shape)
print(old_points.shape)

(472, 3)
(1562, 3)


In [27]:
import torch
import numpy as np

landmarks = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
landmarks = np.array(
    [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
)

if not isinstance(landmarks, torch.Tensor):
    landmarks = torch.tensor(landmarks)


if landmarks.is_floating_point():
    dtype = landmarks.dtype
    if torch.all(landmarks == landmarks.long().to(dtype=dtype)):
        landmarks = landmarks.long()

landmarks

/tmp/ipykernel_177429/1833343818.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  landmarks = torch.tensor(c)


tensor([[1, 2, 3],
        [4, 5, 6],
        [1, 2, 3],
        [4, 5, 6]])

In [1]:
"""3D slider to move a mesh interactively"""
from vedo import Plotter, Mesh, dataurl, settings

settings.default_backend = "vtk"


plt = Plotter()

mesh = Mesh(dataurl + "spider.ply")
mesh.normalize().rotate_z(190)


def slider_y(widget, event):
    mesh.y(widget.value)  # set y coordinate position


plt.add_slider3d(
    slider_y,
    pos1=[0.5, -3.5, 0.35],
    pos2=[0.5, -1.0, 0.35],
    xmin=-1,
    xmax=1,
    value=0,
    s=0.04,
    c="r",
    rotation=45,
    title="y position",
)

plt.show(mesh, __doc__, axes=11, bg="bb", bg2="navy")
plt.close()

: 

: 